# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
# TODO: Implement load the data here.
import pickle
training_file = 'data/train.p'
testing_file = 'data/test.p'
more_data_file = 'data/more_data.p'
checkpoint_file = 'checkpoints/ipnb_checkpoint.chk'
abort_file = 'abort'

FEATURES = 'features'
LABELS = 'labels'
SIZES = 'sizes'
COORDS = 'coords'
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

X_train, y_train, sizes_train, coords_train = train[FEATURES], train[LABELS], train[SIZES], train[COORDS]
X_test, y_test, sizes_test, coords_test = test[FEATURES], test[LABELS], test[SIZES], test[COORDS]


# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [2]:
import numpy as np
import cv2

# TODO: Implement data normalization here.
x_train_norm = np.zeros_like(X_train, dtype=np.float32)
x_test_norm = np.zeros_like(X_test, dtype=np.float32)
for i in range(0, len(X_train)):
    cv2.normalize(X_train[i], x_train_norm[i], -0.5, 0.5, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
for i in range(0, len(X_test)):
    cv2.normalize(X_test[i], x_test_norm[i], -0.5, 0.5, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
X_train = x_train_norm
X_test = x_test_norm

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
#assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [13]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

batch_size = 128
nb_classes = max(y_train) + 1
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 32, 32

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# TODO: Build a two-layer feedforward neural network with Keras here.
model = Sequential()
model.add(Flatten(batch_input_shape=(None,32,32,3)))
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name='output'))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

X_train shape: (39209, 32, 32, 3)
39209 train samples
12630 test samples


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [4]:
# TODO: Compile and train the model here.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=2, verbose=1)

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

Epoch 1/2
39209/39209 [==============================] - 2s - loss: 1.6321 - acc: 0.5524     
Epoch 2/2
39209/39209 [==============================] - 2s - loss: 0.7307 - acc: 0.7856     


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [16]:
from sklearn.model_selection import train_test_split

# TODO: Split some of the training data into a validation dataset.
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)

# TODO: Compile and train the model to measure validation accuracy.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_trainval, Y_trainval, batch_size=batch_size, nb_epoch=20, verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_trainval.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/20
29406/29406 [==============================] - 2s - loss: 0.3315 - acc: 0.8990 - val_loss: 0.2203 - val_acc: 0.9470
Epoch 2/20
29406/29406 [==============================] - 1s - loss: 0.3106 - acc: 0.9043 - val_loss: 0.2116 - val_acc: 0.9486
Epoch 3/20
29406/29406 [==============================] - 1s - loss: 0.2935 - acc: 0.9098 - val_loss: 0.1995 - val_acc: 0.9512
Epoch 4/20
29406/29406 [==============================] - 1s - loss: 0.2874 - acc: 0.9112 - val_loss: 0.2044 - val_acc: 0.9480
Epoch 5/20
29406/29406 [==============================] - 2s - loss: 0.2726 - acc: 0.9159 - val_loss: 0.1776 - val_acc: 0.9562
Epoch 6/20
29406/29406 [==============================] - 2s - loss: 0.2634 - acc: 0.9178 - val_loss: 0.1774 - val_acc: 0.9595
Epoch 7/20
29406/29406 [==============================] - 2s - loss: 0.2605 - acc: 0.9212 - val_loss: 0.1895 - val_acc: 0.9563
Epoch 8/20
29406/29406 [==============================] - 2s -

**Validation Accuracy**: ~96-97%%

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [6]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.

# number of convolutional filters to use
nb_filters = 32
# convolution kernel size
kernel_size = (3, 3)
# size of pooling area for max pooling
pool_size = (2, 2)

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=(img_rows, img_cols, 3)))
model.add(Activation('relu'))
#model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

model.add(Flatten()) # batch_input_shape=(None,32,32,3)
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name='output'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

from sklearn.model_selection import train_test_split
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)
model.fit(X_trainval, Y_trainval, batch_size=batch_size, nb_epoch=10, verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 29s - loss: 1.8005 - acc: 0.5276 - val_loss: 0.6816 - val_acc: 0.8353
Epoch 2/2
29406/29406 [==============================] - 30s - loss: 0.6744 - acc: 0.8125 - val_loss: 0.3108 - val_acc: 0.9326


**Validation Accuracy**: 92.81%

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [6]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.

# number of convolutional filters to use
nb_filters = 32
# convolution kernel size
kernel_size = (3, 3)
# size of pooling area for max pooling
pool_size = (2, 2)

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=(img_rows, img_cols, 3)))
model.add(Activation('relu'))
#model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
#model.add(Dropout(0.25))

model.add(Flatten()) # batch_input_shape=(None,32,32,3)
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name='output'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

from sklearn.model_selection import train_test_split
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)
model.fit(X_trainval, Y_trainval, batch_size=batch_size, nb_epoch=10, verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/10
29406/29406 [==============================] - 18s - loss: 2.0461 - acc: 0.4545 - val_loss: 0.7987 - val_acc: 0.8211
Epoch 2/10
29406/29406 [==============================] - 18s - loss: 0.7719 - acc: 0.7804 - val_loss: 0.3795 - val_acc: 0.9239
Epoch 3/10
29406/29406 [==============================] - 19s - loss: 0.4797 - acc: 0.8653 - val_loss: 0.2520 - val_acc: 0.9461
Epoch 4/10
29406/29406 [==============================] - 19s - loss: 0.3545 - acc: 0.9008 - val_loss: 0.1846 - val_acc: 0.9652
Epoch 5/10
29406/29406 [==============================] - 19s - loss: 0.2856 - acc: 0.9210 - val_loss: 0.1621 - val_acc: 0.9615
Epoch 6/10
29406/29406 [==============================] - 19s - loss: 0.2389 - acc: 0.9348 - val_loss: 0.1266 - val_acc: 0.9730
Epoch 7/10
29406/29406 [==============================] - 19s - loss: 0.2026 - acc: 0.9440 - val_loss: 0.1183 - val_acc: 0.9759
Epoch 8/10
29406/29406 [==============================]

**Validation Accuracy**: ~97-98%

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [8]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.

# number of convolutional filters to use
nb_filters = 32
# convolution kernel size
kernel_size = (3, 3)
# size of pooling area for max pooling
pool_size = (2, 2)

model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=(img_rows, img_cols, 3)))
model.add(Activation('relu'))
#model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
#model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten()) # batch_input_shape=(None,32,32,3)
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name='output'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

from sklearn.model_selection import train_test_split
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)
model.fit(X_trainval, Y_trainval, batch_size=batch_size, nb_epoch=2, verbose=1, validation_data=(X_val, Y_val))

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 23s - loss: 2.2693 - acc: 0.3997 - val_loss: 0.9780 - val_acc: 0.7757
Epoch 2/2
29406/29406 [==============================] - 23s - loss: 0.9951 - acc: 0.7131 - val_loss: 0.4946 - val_acc: 0.8814


**Validation Accuracy**: 91.3%

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [ ]:
#######################################
#      OPtimizing the Network         #
#######################################

input_shapes = [(32, 32, 3), None, None]
nb_filters = [32, 64, 128]
kernel_sizes = [(3, 3), (3, 3), (3, 3)]
pool_sizes = [(2, 2), (3, 3), None]
relus = [True, True, True]
dropouts = [1, 0.5, None]

model = Sequential()
for i in range (len(nb_filters)):
    if not input_shapes[i] == None:
        model.add(Convolution2D(nb_filters[i], kernel_sizes[i][0], kernel_sizes[i][1], border_mode='valid', input_shape=(img_rows, img_cols, 3)))
    else:
        model.add(Convolution2D(nb_filters[i], kernel_sizes[i][0], kernel_sizes[i][1], border_mode='valid'))

    if relus[i] == True:
        model.add(Activation('relu'))

    if not pool_sizes[i] == None:
        model.add(MaxPooling2D(pool_size=pool_sizes[i]))
        
    if not dropouts[i] == None:
        model.add(Dropout(dropouts[i]))

model.add(Flatten()) # batch_input_shape=(None,32,32,3)
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax', name='output'))

model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

from sklearn.model_selection import train_test_split
X_trainval, X_val, Y_trainval, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)
model.fit(X_trainval, Y_trainval, batch_size=batch_size, nb_epoch=10, verbose=1, validation_data=(X_val, Y_val))



Train on 29406 samples, validate on 9803 samples
Epoch 1/10
 3712/29406 [==>...........................] - ETA: 26s - loss: 3.6695 - acc: 0.0533

**Best Validation Accuracy:** (fill in here)

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [19]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Evaluation score:', score[0])
print('Evaluation accuracy:', score[1])

#model.evaluate(X_test, Y_test)

Evaluation score: 0.434374346359
Evaluation accuracy: 0.879255740263


**Test Accuracy:** (fill in here)

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.